Copyright (c) 2020. Cognitive Scale Inc. All rights reserved.
Licensed under CognitiveScale Example Code [License](https://github.com/CognitiveScale/cortex-certifai-examples/blob/7998b8a481fccd467463deb1fc46d19622079b0e/LICENSE.md)

# Introduction

This fifth notebook shows you how to explore the trust score results in a notebook. If you have not already done so, please run the [fourth notebook](patient-readmission-trust-scan.ipynb) to scan the models for their trust scores.

In this notebook, we will:
1. Load the previously saved trust score reports
2. Convert the results into a dataframe and display them


In [1]:
import numpy as np
import pandas as pd
from pprint import pprint

from certifai.scanner.report_reader import ScanReportReader
from certifai.scanner.report_utils import (
    scores, construct_scores_dataframe, construct_feature_scores_dataframe, construct_overall_scores_dataframe,
    construct_group_scores_dataframe)
from certifai.scanner.builder import ExplanationType
from IPython.display import display, Markdown

# Loading the Trust Score Reports

To load the reports, we need to know the use case ID ('readmission') and the scan ID.

List the available use cases, and the scans within the 'readmission' use case.

In [2]:
reader = ScanReportReader("reports")
reader.list_usecases()
scans = reader.list_scans('readmission')
data=[[s['date'], ', '.join(s['reportTypes']), s['id']] for s in scans]
df = pd.DataFrame(data, columns=['date', 'evals', 'scan id']).sort_values(by=['date'], ascending=False)
print(df)

              date                                      evals       scan id
0  20230106T143234  atx, explainability, fairness, robustness  7e0a4f5e3168
1  20230106T140239                                explanation  025543f8e4a4


Locate the latest trust score scan and load it.

In [3]:
latest_scan = df[df['evals'].str.contains('fairness') & df['evals'].str.contains('robustness')].iloc[0]
results = reader.load_scan('readmission', latest_scan['scan id'])

# Extract the results

In this section we'll construct dataframes containing the score results and display them.

Print out the overall fairness results.

In [4]:
fairness_scores = list(scores('fairness', results))
display(construct_overall_scores_dataframe(fairness_scores, include_confidence=True))

,context,eval type,score type,score
0,mlp,fairness,burden,75.464094
1,logit,fairness,burden,61.007313


Print out the fairness scores by feature

In [5]:
display(construct_feature_scores_dataframe(fairness_scores, include_confidence=True))

,context,eval type,score type,age,age lower bound,age upper bound,gender,gender lower bound,gender upper bound,race,race lower bound,race upper bound
0,mlp,fairness,burden,75.464094,70.206183,80.490767,83.827691,74.931745,93.031928,77.571753,68.967126,85.051824
1,logit,fairness,burden,61.007313,54.470172,67.479765,75.176488,62.948303,87.552332,67.394669,55.574481,78.272413


Print out the detailed fairness scores

In [6]:
display(construct_group_scores_dataframe(fairness_scores, include_confidence=True))

,context,eval type,score type,value type,age - above 95,age - above 95 lower bound,age - above 95 upper bound,age - up to 05,age - up to 05 lower bound,age - up to 05 upper bound,...,race - Caucasian upper bound,race - Hispanic,race - Hispanic lower bound,race - Hispanic upper bound,race - Other,race - Other lower bound,race - Other upper bound,race - nan,race - nan lower bound,race - nan upper bound
0,mlp,fairness,burden,burden,0.047856,0.030541,0.067316,0.009824,0.004795,0.016341,...,0.054469,0.066944,0.047179,0.094781,0.032699,0.021137,0.046125,0.047368,0.022346,0.074074
1,logit,fairness,burden,burden,0.094095,0.067194,0.127176,0.000960,0.000000,0.002951,...,0.044432,0.038516,0.025315,0.055519,0.017527,0.008425,0.029536,0.015306,0.000000,0.032609


For an example of visualizing the fairness results in a chart, see the [fairness metrics notebook](https://github.com/CognitiveScale/cortex-certifai-examples/blob/master/notebooks/fairness_metrics/FairnessMetrics.ipynb).

Print out the robustness scores.

In [7]:
df = construct_scores_dataframe(scores('robustness', results), include_confidence=True)
display(df)

,context,robustness,robustness lower bound,robustness upper bound
logit,logit,80.549076,76.406660,84.236577
mlp,mlp,65.788375,60.572259,70.833897


Print out the explainability scores.

In [8]:
df = construct_scores_dataframe(scores('explainability', results), include_confidence=False)
display(df)

,context,explainability,Num features (1),Num features (10),Num features (2),Num features (3),Num features (4),Num features (5),Num features (6),Num features (7),Num features (8),Num features (9)
logit,logit,97.421875,76.5625,0.0,21.87500,1.56250,0.0,0.0,0.0,0.0,0.0,0.0
mlp,mlp,96.523438,68.7500,0.0,28.90625,2.34375,0.0,0.0,0.0,0.0,0.0,0.0
